In [ ]:
import math
import random
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def polynomial_kernel(x, y, p):
    return pow(np.dot(x,y)+1, p)

def linear_kernel(x, y):
    return np.dot(x,y)

def radial_basis_kernel(sigma, x, y):
    sec1 = np.linalg.norm(np.subtract(x,y))
    sec2 = -((sec1**2) / (2*(sigma**2)))
    return math.exp(sec2)

In [ ]:
def objective(a):
    sec = 0
    for i in range(N):
        for j in range(N):
            sec += 0.5*a[i]*a[j]*target[i]*target[j]*kernel(x[i], x[j])
    return sec - np.sum(a)

In [ ]:
def zerofun(a):
    z=0
    for i in range(len(a)):
        z += a[i]*target[i]
        return z

In [ ]:
def judge_non_zero(n):
    if abs(n) > 0.00001:
        return True
    else:
        return False

In [ ]:
def b_value(data):
    sec = 0
    for i in range(len(data)):
        sec += data[i][0][0]*data[i][1][0]*kernel(data[i][2],data[0][2])
    b = sec-data[0][0][0]
    return b

In [ ]:
def indicator(x, data):
    sec = 0
    for i in range(len(data)):
        sec += data[i][0][0]*data[i][1][0]*kernel(x,data[i][2])
    return sec-b_value(data)

In [ ]:
def random_data(n, d, s, u):
    return s * np.random.randn(n, d) + u

In [ ]:
def generate_data(d, ptr):
    # ptr : for [data_size, standard deviation, average[0], average[1]] in 3
    
    classA = np.concatenate((random_data(ptr[0][0],d, ptr[0][1], [ptr[0][2],ptr[0][3]]),
                                random_data(ptr[1][0],d, ptr[1][1], [ptr[1][2],ptr[1][3]])))
    
    classB = np.concatenate((random_data(ptr[2][0],d, ptr[2][1], [ptr[2][2],ptr[2][3]]),
                                random_data(ptr[3][0],d, ptr[3][1], [ptr[3][2],ptr[3][3]])))
    inputs = np.concatenate((classA, classB))
    targets = np.concatenate((
        np.ones(classA.shape[0]),
        -np.ones(classB.shape[0])
    ))
    N = inputs.shape[0] ## Number of rows ie. samples
    permute = list(range(N))
    random.shuffle(permute)
    inputs = inputs[permute, :]
    targets = targets[permute]
    return (inputs, targets, classA, classB)

In [ ]:
def kernel(x, y):
    # return linear_kernel(x, y)
    # return polynomial_kernel(x, y, 7)
    return radial_basis_kernel(0.3, x, y)

In [ ]:
data_shape = [[ 10 , 0.2 , 1.5 , 0.5 ],
              [ 10 , 0.2, -1.5 , 0.5 ],
              [ 10 , 0.2 , 0.0 , 0.5 ],
              [ 10 , 0.2 , 0.0 , 0.5 ]]
D = 2
C = 10

In [ ]:
# random.seed(100)
x,target, classA, classB = generate_data(D, data_shape)
N = len(x)

In [ ]:
start = np.zeros(N)
sv = []
ret = minimize(objective, start, bounds=[(0, C) for b in range(N)],constraints={'type':'eq', 'fun':zerofun})
if not ret.success:
    print("Optimization was unsuccessful")
alpha = ret['x']
for i in range(len(alpha)):
    if judge_non_zero:
        sv.append([[alpha[i]],[target[i]],[x[i][0], x[i][1]]])
b = b_value(sv)

In [ ]:
plt.plot([p[0] for p in classA], [p[1] for p in classA],'b.')
plt.plot([p[0] for p in classB], [p[1] for p in classB],'r.')
plt.axis('equal')
xgrid=np.linspace(-5,5)
ygrid=np.linspace(-4,4)
grid=np.array([[indicator([x,y], sv) for x in xgrid ] for y in ygrid])
plt.contour(xgrid, ygrid, grid, (-1.0,0.0,1.0), colors =('red' ,'black' , 'blue'),linewidths=(1 , 3 , 1))
plt.show()